In [16]:
import bdrk
import lightgbm as lgb
import logging
import numpy as np
import os
import pandas as pd
import pickle
import sys

from bedrock_client.bedrock.analyzer.model_analyzer import ModelAnalyzer
from bedrock_client.bedrock.analyzer import ModelTypes
from sklearn import metrics
from utils import lgb_roc_auc_score, print_results

%matplotlib inline
%load_ext autoreload
%autoreload 2


# You can customize the logging logic here
_logger = logging.getLogger(bdrk.utils.vars.Constants.MAIN_LOG)
_logger.setLevel(logging.INFO)
if not _logger.handlers:
    _logger.addHandler(logging.StreamHandler(stream=sys.stdout))
    
# Visit https://bedrock.basis-ai.com/setting/token/ to get the personal access token.
with open('bdrk_token') as f:
    token = f.read().strip("\n")

os.environ["BEDROCK_API_TOKEN"] = token


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data processing

In [17]:
# Subscribers data
subscribers = pd.read_parquet("data/subscribers.gz.parquet", engine="fastparquet")
for col in ["Churn", "Intl_Plan", "VMail_Plan"]:
    subscribers[col] = (subscribers[col] == "yes").astype(int)

# Calls data
all_calls = pd.read_parquet("data/all_calls.gz.parquet", engine="fastparquet")
all_calls = all_calls.groupby(["Call_type", "User_id"]).agg(["sum", "count"]).reset_index()
all_calls.columns = ["Call_type", "User_id", "Mins", "Calls"]
all_calls = all_calls.pivot(index="User_id", columns="Call_type", values=["Mins", "Calls"]).reset_index()
all_calls.columns = [
    "User_id", "Day_Mins", "Eve_Mins", "Intl_Mins", "Night_Mins",
    "Day_Calls", "Eve_Calls", "Intl_Calls", "Night_Calls"]

# Joining
model_data = pd.merge(subscribers, all_calls, on=["User_id"], how="left")
model_data = model_data.fillna(0)

# Categorical encoding
model_data = pd.get_dummies(model_data, columns=["Area_Code", "State"])

In [18]:
FEATURE_COLS = [
    "Intl_Plan", "VMail_Plan", "VMail_Message", "CustServ_Calls",
    "Day_Mins", "Day_Calls", "Eve_Mins", "Eve_Calls", 
    "Night_Mins", "Night_Calls", "Intl_Mins", "Intl_Calls",
    "Area_Code_408", "Area_Code_415", "Area_Code_510",
    "State_AK", "State_AL", "State_AR", "State_AZ", "State_CA", "State_CO",
    "State_CT", "State_DC", "State_DE", "State_FL", "State_GA", "State_HI",
    "State_IA", "State_ID", "State_IL", "State_IN", "State_KS", "State_KY",
    "State_LA", "State_MA", "State_MD", "State_ME", "State_MI", "State_MN",
    "State_MO", "State_MS", "State_MT", "State_NC", "State_ND", "State_NE",
    "State_NH", "State_NJ", "State_NM", "State_NV", "State_NY", "State_OH",
    "State_OK", "State_OR", "State_PA", "State_RI", "State_SC", "State_SD",
    "State_TN", "State_TX", "State_UT", "State_VA", "State_VT", "State_WA",
    "State_WI", "State_WV", "State_WY",
]

TARGET_COL = "Churn"

## Train/test split

In [19]:
train_data, eval_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))])

X_train = train_data[FEATURE_COLS]
y_train = train_data[TARGET_COL]

X_val = eval_data[FEATURE_COLS]
y_val = eval_data[TARGET_COL]

X_test = test_data[FEATURE_COLS]
y_test = test_data[TARGET_COL]

# Training

## Training parameter

In [20]:
LR = 0.05
NUM_LEAVES = 10
N_ESTIMATORS = 100
OUTPUT_MODEL = "data/lgb_model.pkl"

## Training code

In [21]:
# Initialize the project
bdrk.init(project_id="nglinh-churn-prediction")

with bdrk.start_run(pipeline_id="nglinh-new-pipeline", environment_id="playground-production"):
    # Log related parameters
    bdrk.log_params({"LR": LR, "NUM_LEAVES": NUM_LEAVES, "N_ESTIMATORS": N_ESTIMATORS})
    
    
    clf = lgb.LGBMClassifier(num_leaves=NUM_LEAVES, learning_rate=LR, n_estimators=N_ESTIMATORS)
    
    # Log a single metric
    bdrk.log_metric(key="Feature Count", value=len(FEATURE_COLS))
    
    clf.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=lgb_roc_auc_score,
        verbose=False,
    )
    
    # Log metrics for each training step
    evals = clf.evals_result_["valid_0"]
    for iteration in range(N_ESTIMATORS):
        bdrk.log_metrics(metrics={
            "Training binary_logloss": evals["binary_logloss"][iteration],
            "Training roc_auc": evals["roc_auc"][iteration]
        }, x=iteration)
        

    y_probs = clf.predict_proba(X_test)[:,1]
        
    
    # Log final metrics in batch
    y_pred = (y_probs > 0.5).astype(int)
    bdrk.log_metrics(metrics={
        "Accuracy": metrics.accuracy_score(y_test, y_pred),
        "Precision": metrics.precision_score(y_test, y_pred),
        "Recall": metrics.recall_score(y_test, y_pred),
        "F1 score": metrics.f1_score(y_test, y_pred),
        "Roc auc": metrics.roc_auc_score(y_test, y_pred),
        "Avg precision": metrics.average_precision_score(y_test, y_pred),
    })
    
    # Log the binary classifier from the predicted data
    bdrk.log_binary_classifier_metrics(actual=y_test, probability=y_probs)

    # Saving and log the model
    with open(OUTPUT_MODEL, "wb") as model_file:
        pickle.dump(clf, model_file)
    bdrk.log_model(OUTPUT_MODEL)


BedrockClient initialized on project=nglinh-churn-prediction


INFO:bdrk:BedrockClient initialized on project=nglinh-churn-prediction


Run started: nglinh-new-pipeline-run16


INFO:bdrk:Run started: nglinh-new-pipeline-run16


Params updated: {'LR': '0.05', 'NUM_LEAVES': '10', 'N_ESTIMATORS': '100'}


...

INFO:bdrk:Model version created: nglinh-new-pipeline-v9


Run Succeeded: nglinh-new-pipeline-run16


INFO:bdrk:Run Succeeded: nglinh-new-pipeline-run16


Run exitted


INFO:bdrk:Run exitted


In [22]:
print_results(y_test, y_probs)

Confusion matrix:
[[282   4]
 [ 10  38]] 

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       286
           1       0.90      0.79      0.84        48

    accuracy                           0.96       334
   macro avg       0.94      0.89      0.91       334
weighted avg       0.96      0.96      0.96       334

